In [6]:
from qdrant_client import QdrantClient, models

In [7]:
client = QdrantClient(url="http://localhost:6333")

In [8]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2, 3, 4, 5, 6],
        payloads=[
            {"color": "red", "city": "London",},
            {"color": "green", "city": "Moscow"},
            {"color": "blue", "city": "Moscow"},
            {"color": "green", "city": "London"},
            {"color": "red", "city": "Berlin"},
            {"color": "blue", "city": "London"},
        ],
        vectors=[
            [0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2],
            [0.3, 0.3, 0.3],
            [0.4, 0.4, 0.4],
            [0.5, 0.5, 0.5],
            [0.6, 0.6, 0.6],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="color",
                # exact match of the given value
                match=models.MatchValue(value="red"),  # 用于关键字、整数、布尔值
            )
        ]
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None)],
 None)

In [11]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="color",
                # In case you want to check if the stored value is one of multiple values, you can use the Match Any condition. 
                match=models.MatchAny(any=["red", "blue"]),  # 用于关键字、整数
            )
        ]
    ),
)

([Record(id=1, payload={'city': 'London', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'Moscow', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
  Record(id=5, payload={'city': 'Berlin', 'color': 'red'}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)

In [13]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="color",
                # In case you want to check if the stored value is not one of multiple values, you can use the Match Except condition. 
                match=models.MatchExcept(**{"except": ["red", "black"]}), # 用于关键字、整数
            )
        ]
    ),
)

([Record(id=2, payload={'city': 'Moscow', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=3, payload={'city': 'Moscow', 'color': 'blue'}, vector=None, shard_key=None, order_value=None),
  Record(id=4, payload={'city': 'London', 'color': 'green'}, vector=None, shard_key=None, order_value=None),
  Record(id=6, payload={'city': 'London', 'color': 'blue'}, vector=None, shard_key=None, order_value=None)],
 None)